## 导入库

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

## 超参

In [3]:
embed_size = 300
max_features = 50000
maxlen = 100

## 导入数据 

In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [4]:
train_X = train_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [7]:
train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=maxlen)
test_X = keras.preprocessing.sequence.pad_sequences(test_X, maxlen=maxlen)

In [8]:
train_y = train_df['target'].values

## 模型

In [13]:
model = keras.models.Sequential([
    keras.layers.Embedding(max_features, embed_size),
    keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64, return_sequences=True)),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 300)         15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         187392    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 15,189,473
Trainable params: 15,189,473
Non-trainable params: 0
____________________________________________

## 损失和优化 

In [16]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])

## 开始训练 

In [17]:
history = model.fit(train_X, train_y, batch_size=512, epochs=3, validation_split=0.2)

Train on 1044897 samples, validate on 261225 samples
Epoch 1/3
1044897/1044897 [==============================] - 355s 340us/sample - loss: 0.1284 - accuracy: 0.9502 - val_loss: 0.1144 - val_accuracy: 0.9552
Epoch 2/3
1044897/1044897 [==============================] - 346s 332us/sample - loss: 0.1009 - accuracy: 0.9596 - val_loss: 0.1093 - val_accuracy: 0.9567
Epoch 3/3
1044897/1044897 [==============================] - 346s 331us/sample - loss: 0.0868 - accuracy: 0.9657 - val_loss: 0.1184 - val_accuracy: 0.9565


## 保存

In [18]:
model.save('./save/save_model/model.h5')

In [19]:
model.save_weights('./save/save_weights/model_weights.h5')